# Laboratorio #3: Expansión de Consulta y Evaluación de Modelos
 
En esta clase se abordarán conceptos fundamentales sobre la mejora de la recuperación mediante técnicas de expansión de consultas y cómo evaluar la efectividad de los modelos utilizando métricas estándar. 

Para mejorar los resultados de una búsqueda, una técnica común es la **expansión de la consulta**. Esto implica agregar términos relacionados semánticamente a la consulta original. Existen distintos tipos de métodos y técnicas para expandir la consulta efectuada por un usuario. Los más comunes son:

1. Reformulación de la consula (sinónimos y términos relacionados, expansión manual).
2. Retroalimentación (retroalimentación relevante, pseudo-relevancia).
3. Modelos de Lenguaje y Procesamiento del Lenguaje Natural (modelos de lenguaje, word embeddings).
4. Expansión basada en la búsqueda (exploración de términos, consulta dividida).

In [ ]:
# Procesamiento de lenguaje natural
import nltk

# Facilita el trabajo con sinónimos y antónimos  
from nltk.corpus import wordnet

# Asegurar que WordNet esté descargado
try:
    wordnet.synsets("test")  
    print("WordNet: Ready to use")
except LookupError:
    nltk.download('wordnet')
    nltk.download("wordnet")
    nltk.download("omw-1.4")
except Exception as e:
    print(f"Download error. WordNet: {e}")
    
# Catálogo de datsets
import ir_datasets

# Facilita la tokenización de los documentos y la visualización de los resultados
from teacher_help import tokenize, plot_ir_confusion_matrix

# Facilita el procesamiento de un corpus de texto
from gensim.corpora.dictionary import Dictionary

### Matriz de co-ocurrencia

La expansión de consulta con matriz de co-ocurrencia consiste en identificar términos que aparecen frecuentemente junto a los términos de la consulta en los documentos del corpus. La matriz de co-ocurrencia almacena estas relaciones, y al analizarla podemos añadir a la consulta original aquellos términos que más co-ocurren con ella, mejorando así la recuperación de documentos relevantes.


## **Ejercicio #1:** Implemente la función `co_ocurrence_matrix` para obtener la matriz de co-ocurrencia. 

###### Ayuda #1: Para esto cargue el corpus Cranfield y tokenice los documentos para luego instanciar la clase *Dictionary* de gensim.

In [ ]:
# TODO

In [ ]:

def co_ocurrence_matrix(docs, dictionary, window_size=5):
    """
    Función encargada de construir la matriz de co-ocurrencia asociada a un corpus.
    Args:
        docs (list<(str, [str])>): Lista de documentos con los que se construirá la matriz de co-ocurrencia.
        dictionary (Dictionary): Instancia de la clase Dictionary construída a partir de los documentos con los que se construirá la matriz de co-ocurrencia.
        window_size (int, optional): Distancia que se tendrá en cuanta para asociar a una palabra con otro en la matriz de co-ocurrencia. Defaults to 5.

    Returns:
        [[int]]: Matriz de co-ocurrencia.
    """
    
    # TODO

id_token_0 = dictionary.token2id['experimental']
id_token_1 = dictionary.token2id['study']

matrix = co_ocurrence_matrix(tokenized_corpus, dictionary)
print(f"Co-ocurrencia asociada a las palabras experimental y study: {matrix[id_token_0][id_token_1]}")

### Wordnet

Una herramienta útil es **WordNet**, una base de datos léxica para el inglés incluida en la biblioteca `nltk` (Natural Language Toolkit). WordNet organiza palabras en conjuntos similares (synsets) y proporciona relaciones semánticas entre ellas, de ahí que es utilizado para la labor de expansión de consultas.

> ⚠️ Sin embargo, no basta con añadir palabras relacionadas con la consulta original: a menudo es necesaria la **estructuración de la consulta**.

Al expandir una consulta, se agregan términos relacionados que pueden ayudar a recuperar más resultados. Sin embargo, esto también puede generar ruido, ya que todos los términos tienen el mismo peso, incluso los menos relevantes. Además, puede aumentar la ambigüedad y dificultar la interpretación del sistema.

Por eso, es útil estructurar la consulta utilizando operadores booleanos (AND, OR, NOT), agrupaciones con paréntesis y filtros como rangos de fechas. Esto permite mantener el foco del usuario, mejorar la precisión de los resultados y controlar mejor el comportamiento del sistema. De esta forma luego de la expansion y la estructuración la consulta:

`impacto del cambio climático en la biodiversidad marina`

se convertiría en 

`("cambio climático" AND "biodiversidad marina") AND ("impacto" OR "efecto" OR "consecuencia") NOT ("política" OR "economía") AND (publicado:2015..2024)`

## **Ejercicio #2**: Implemente una función que dada una consulta en lenguaje natural, la expanda usando sinónimos obtenidos desde WordNet.

###### Ayuda #1: Tenga en cuenta que en ocasiones los en wordnet los 'sinónimos' están constituídos por dos palabras separadas por un guión bajo (_).

In [ ]:
def expand_query_with_synonyms(query):
    """
    Expande una consulta utilizando sinónimos de WordNet.
    Args:
        query (str): La consulta original.
    Returns:
        dict: Un diccionario con términos originales y sus sinónimos.
    """
    # TODO

query = "information retrieval"
expanded = expand_query_with_synonyms(query)

print("Consulta original:", query)
print("Términos de la consulta expandida:", expanded)

## **Ejercicio #3:** Implemente además la estructuración de la consulta.

In [ ]:
def structure_expanded_query(expanded_terms):
    """
    Estructura la consulta expandida en un formato lógico.
    Args:
        expanded_terms (dict): Un diccionario con términos originales y sus sinónimos.

    Returns:
        str: La consulta estructurada en formato lógico.
    """
    # TODO

print("Consulta original:", query)
structured = structure_expanded_query(expanded)
print("Consulta expandida y estructurada:", structured)

### Evaluemos entonces los resultados del modelo luego de aplicar la expansión de consultas: **medidas de evaluación**

Las medidas de evaluación desempeñan un papel fundamental en los Sistemas de Recuperación de Información (SRI) al proporcionar una forma objetiva de medir su rendimiento y efectividad. Estas medidas permiten a las personas comprender y mejorar la calidad de los resultados recuperados por el sistema.

Como importancia del uso de las medidas de evaluación, se pueden mencionar:

1. **Calidad de los Resultados**: Las medidas de evaluación permiten determinar la precisión y relevancia de los documentos recuperados en relación con las consultas de los usuarios. Cuanto más precisos y relevantes sean los resultados, mayor será la utilidad y satisfacción del usuario.

2. **Optimización del Sistema**: Al proporcionar métricas cuantitativas, las medidas de evaluación permiten a los desarrolladores identificar áreas de mejora en el SRI. Esto puede implicar ajustes en algoritmos de búsqueda, ponderación de términos, o mejoras en la indexación de documentos, entre otros.

3. **Comparación de Métodos y Sistemas**: Las medidas de evaluación facilitan la comparación entre diferentes métodos de búsqueda y sistemas. Esto es fundamental para seleccionar la mejor solución en función de los requisitos específicos del usuario y del contexto de aplicación.

No obstante, crear los conjuntos de datos para aplicar las distintas métricas y evaluar el sistema es un laborioso trabajo que solo puede ser llevado por expertos en el tema.

> 😉 Continuaremos trabajando con los documentos del corpus **Cranfield**.

### Usando las métricas de sklearn 

Scikit-learn, comúnmente conocido como `sklearn`, es una biblioteca de software de código abierto que proporciona herramientas simples y eficientes para el análisis de datos y la modelación estadística. Esta biblioteca es ampliamente utilizada en el campo del aprendizaje automático para la implementación de algoritmos de clasificación, regresión, agrupamiento y reducción de dimensionalidad. Además, `sklearn` puede utilizarse para medir el desempeño de un SRI a través de métricas de evaluación como precisión, recobrado y F1, o para obtener la matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

### Matriz de Confusión

La matriz de confusión es una herramienta de análisis que permite la evaluación del desempeño de un sistema de clasificación. Se estructura en cuatro componentes esenciales:

* Verdaderos Positivos (TP): Número de elementos correctamente identificados como relevantes.
* Falsos Positivos (FP): Número de elementos incorrectamente identificados como relevantes.
* Verdaderos Negativos (TN): Número de elementos correctamente identificados como no relevantes.
* Falsos Negativos (FN): Número de elementos incorrectamente identificados como no relevantes.

Esta matriz proporciona una base para calcular otras métricas de evaluación de desempeño.

In [ ]:
# Etiquetas verdaderas
#   0 -> documento relevante
#   1 -> documento irrelevante
y_true = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1]  

# Predicciones del modelo SRI
#   0 -> documento recuperado
#   1 -> documento no recuperado
y_pred = [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]  # Predicciones del modelo

labels = ['relevant', 'irelevant']

matrix = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = matrix.ravel()

print('Matrix de confusión: \n', matrix)
print('')
print('Cantidad de verdaderos positivos (TP):', tp)
print('Cantidad de falsos positivos (FP):', fp)
print('Cantidad de falsos negativos (FN):', fn)
print('Cantidad de verdaderos negativos (TN):', tn)

plot_ir_confusion_matrix(tp, fp, fn, tn)

## Precisión

La precisión indica qué proporción de los elementos recuperados son relevantes para la consulta realizada y se calcula como:
$$ P = \frac{|TP|}{|TP| + |FP|} $$

In [ ]:
precision = precision_score(y_true, y_pred)
print(f"Precisión: {precision}")

**Nota:** Una precisión de 1 no necesariamente implica que el sistema haya recuperado todos los elementos relevantes disponibles. La precisión se enfoca exclusivamente en la calidad de los resultados recuperados, no en la completitud. Por lo tanto, un sistema podría tener una precisión perfecta pero un recobrado bajo si solo recupera una pequeña fracción de todos los datos relevantes disponibles. 

## Recobrado

El recobrado representa la proporción de elementos relevantes que fueron recuperados con respecto a todos los elementos relevantes existentes en el conjunto de datos, a partir de la consulta. Se calcula como:
$$ \text{R} = \frac{|TP|}{|TP| + |FN|} $$

In [ ]:
recall = recall_score(y_true, y_pred)
print(f"Recobrado: {recall}")

**Nota:** Un recobrado de 1 no indica nada sobre la cantidad de elementos no relevantes que también puedan haber sido recuperados (esto sería evaluado por la precisión). Por lo tanto, es posible tener un recobrado de 1 con una precisión baja si el sistema recupera muchos elementos irrelevantes junto con todos los relevantes. 

## Medida F

La medida F es una métrica que combina precisión y recobrado en un solo valor, lo que proporciona una evaluación equilibrada del sistema. Se calcula como la media armónica de estas medidas:

$$ F = \frac{(1 + \beta^2) \cdot \text{P} \cdot \text{R}}{\beta^2 \cdot \text{P} + \text{R}} $$

donde **β** es un factor el cual indica la medida a prestarle más atención o importancia.

Cuando:
- $\beta > 1$, el recobrado se considera más importante que la precisión. Por ejemplo, $F_2$ pone más énfasis en el recobrado.
- $\beta < 1$, la precisión se considera más importante. Por ejemplo, $F_{0.5}$ pone más énfasis en la precisión.
- $\beta = 1$, la precisión y el recobrado se consideran igualmente importantes, conociéndose esta como medida $F_1$.

### Medida F1

$$ F1 = \frac{2 \cdot \text{P} \cdot \text{R}}{\text{P} + \text{R}} $$

In [ ]:
f1 = f1_score(y_true, y_pred)
print(f"Medida F1: {f1}")

**Nota:** La razón por la que se utiliza la media armónica en lugar de la media aritmética es que la media armónica penaliza más fuertemente los valores extremos. Esto significa que si uno de los dos valores (precisión o recobrado) es muy bajo, la medida F1 también será baja, lo que refleja una necesidad de equilibrio entre capturar todos los elementos relevantes (recobrado alto) y asegurar que los elementos recuperados sean relevantes (alta precisión).

Luego, sklearn provee la mayoría de las medidas necesarias pero tiene un problema: necesita todo el conjunto de datos clasificado en recuperado, no recuperado, relevante e irrelevante. Esto es un problema en cuanto al espacio de memoria cuando el conjunto de datos excede la cifra de los millones, el cual es un número común en sistemas que trabajan, cargan y procesan datos constantemente de Internet.

Por tanto, **se requiere que implemente cada métrica pero optimizadas a los subconjuntos de datos relevantes y datos recuperados**. 

Para ello, se brinda 2 funciones que puede utilizar.

In [ ]:
import random

def relevant_documents_func(query_id : str):
    """
    Devuelve los documentos relevantes dada una consulta. Se basa en los datos de Crandield.
    
    Args: 
      - query_id (str) : Identificador de la consulta.

    Return: 
      (list<str>, str)
    """
    
    for (queryt_id, query_text) in dataset.queries_iter():
        if queryt_id == query_id:
            break
      
    return (
        [
            doc_id
            for (queryt_id, doc_id, relevance, _) in dataset.qrels_iter()
            if queryt_id == query_id and relevance in [3, 4]
        ], 
        query_text)


def recovered_documents_sri(query):
    """
    Determina el conjunto de los documentos recuperados. El documento más relevante se encuentra en la posición 0 (orden decreciente).

    Args:
      - query (str): Texto de la consulta.

    Return:
      list: Lista de los identificadores de los documentos
    """
    
    document_identifiers = [t[0] for t in dataset.docs_iter()]
    random.shuffle(document_identifiers)
    recovered_documents = document_identifiers[:random.randint(1, len(document_identifiers) - 1)]
    
    return recovered_documents

**PD #1:** La función `relevant_documents` retorna la información provista directamente de ir-dataset.

**PD #2:** La función `recovered_documents_sri` retorna la información aleatoria tomada del conjunto de datos. Acá lo que realmente va es el retorno de su sistema (booleano, vectorial, otro). Como hay que rescatar soluciones pasadas, a efectos de la clase se trabajará con esta implementación tonta.

## **Ejercicio #4:** Implemente cada una de las métricas que a continiación se definen.

###### Ayuda #1: Utilice las operaciones entre conjuntos.

In [ ]:
def precision(recovered_documents, relevant_documents):
    """
    Calcula la medida de precisión.
    
    Args:
      - recovered_documents (list): Conjunto de documentos recuperado por el SRI. Cada documento está representado por su identificador.
      - relevant_documents (list): Conjunto de documentos relevantes. Cada documento está representado por su identificador. 
        
    Return:
      double: Valor entre 0 y 1.    
    """
    
    # TODO


In [ ]:
def recall(recovered_documents, relevant_documents):
    """
    Calcula la media de recobrado.
    
    Args:
      - recovered_documents (list): Conjunto de documentos recuperado por el SRI. Cada documento está representado por su identificador.
      - relevant_documents (list): Conjunto de documentos relevantes. Cada documento está representado por su identificador.

    Return:
      double: Valor entre 0 y 1.     
    """
    
    # TODO


In [ ]:
def f1(recovered_documents, relevant_documents):
    """
    Calcula la medida F1.
    
    Args:
      - recovered_documents (list): Conjunto de documentos recuperado por el SRI. Cada documento está representado por su identificador.
      - relevant_documents (list): Conjunto de documentos relevantes. Cada documento está representado por su identificador.

    Return:
      double: Valor entre 0 y 1.     
    """
      
    # TODO


Para poder verificar las funciones, tome una consulta y determine por su sistema el conjunto de documentos recuperados. Luego, utilice la función siguiente para el apoyo visual.

In [ ]:
query_id = '1'
relevant_documents, query_text = relevant_documents_func(query_id)
recovered_documents = recovered_documents_sri(query_text)

print(f"""
Identificador de la consulta: {query_id}
Consulta: {query_text}

Métricas:
  Precisión: {precision(recovered_documents, relevant_documents)}
  Recobrado: {recall(recovered_documents, relevant_documents)}
  F1: {f1(recovered_documents, relevant_documents)}
""")

### Ejercicio pendiente:
Implemente el resto de las métricas vistas en conferencia, utilizando solo las operaciones entre conjuntos.